In [ ]:
!pip install pyspark py4j

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
spark=SparkSession.builder.appName("Tezos Big Data").getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving balance_updates.csv to balance_updates.csv
Saving blocks.csv to blocks.csv
Saving delegation_operation.csv to delegation_operation.csv
Saving operation.csv to operation.csv
Saving transaction_operation.csv to transaction_operation.csv


In [ ]:
df_operation = spark.read.csv(
    "/content/operation.csv",
    header=True,
    inferSchema=True
)


In [ ]:
df_blocks = spark.read.csv(
    "/content/blocks.csv",
    header=True,
    inferSchema=True
)

In [ ]:
df_delegation_operation = spark.read.csv(
    "/content/delegation_operation.csv",
    header=True,
    inferSchema=True
)

In [ ]:
df_balance_updates = spark.read.csv(
    "/content/balance_updates.csv",
    header=True,
    inferSchema=True
)

In [ ]:
df_transaction_operation = spark.read.csv(
    "/content/transaction_operation.csv",
    header=True,
    inferSchema=True
)

In [ ]:
df_operation.printSchema()
df_operation.show(5)
df_operation.count()

root
 |-- level: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- signature: string (nullable = true)
 |-- operation_hash: string (nullable = true)
 |-- operation_group_index: integer (nullable = true)
 |-- operation_index: integer (nullable = true)
 |-- content_index: integer (nullable = true)
 |-- internal_operation_index: integer (nullable = true)
 |-- operation_kind: string (nullable = true)

+------+-------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------+-------------+------------------------+--------------+
| level|          timestamp|          block_hash|              branch|           signature|      operation_hash|operation_group_index|operation_index|content_index|internal_operation_index|operation_kind|
+------+-------------------+--------------------+--------------------+--------------

28860

In [ ]:
df_delegation_operation.printSchema()
df_delegation_operation.show(5)
df_delegation_operation.count()

root
 |-- level: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- signature: string (nullable = true)
 |-- operation_hash: string (nullable = true)
 |-- operation_group_index: integer (nullable = true)
 |-- operation_index: integer (nullable = true)
 |-- content_index: integer (nullable = true)
 |-- internal_operation_index: string (nullable = true)
 |-- source: string (nullable = true)
 |-- delegate: string (nullable = true)
 |-- fee: integer (nullable = true)
 |-- counter: integer (nullable = true)
 |-- gas_limit: integer (nullable = true)
 |-- storage_limit: integer (nullable = true)
 |-- status: string (nullable = true)

+-------+-------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------+-------------+------------------------+--------------------+--------------------+-----+--------+---------+----

22214

In [ ]:
df_blocks.printSchema()
df_blocks.show(5)
df_blocks.count()

root
 |-- level: integer (nullable = true)
 |-- proto: integer (nullable = true)
 |-- predecessor: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- validation_pass: integer (nullable = true)
 |-- operations_hash: string (nullable = true)
 |-- fitness: string (nullable = true)
 |-- context: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- chain_id: string (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- nonce_hash: string (nullable = true)
 |-- consumed_gas: long (nullable = true)
 |-- baker: string (nullable = true)
 |-- voting_period_kind: string (nullable = true)
 |-- cycle: integer (nullable = true)
 |-- cycle_position: integer (nullable = true)
 |-- voting_period: integer (nullable = true)
 |-- voting_period_position: integer (nullable = true)
 |-- expected_commitment: boolean (nullable = true)
 |-- number_of_operation_groups: integer (nullable = true)
 |-- number_of_operations: integer (nullable = true)

+-------+-----+

20468

In [ ]:
df_balance_updates.printSchema()
df_balance_updates.show(5)
df_balance_updates.count()

root
 |-- level: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- type: string (nullable = true)
 |-- operation_hash: string (nullable = true)
 |-- operation_group_index: integer (nullable = true)
 |-- operation_index: integer (nullable = true)
 |-- content_index: integer (nullable = true)
 |-- internal_operation_index: integer (nullable = true)
 |-- balance_update_index: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- delegate: string (nullable = true)
 |-- change: long (nullable = true)
 |-- category: string (nullable = true)

+------+-------------------+--------------------+------------------+--------------------+---------------------+---------------+-------------+------------------------+--------------------+------+--------+--------------------+--------+----------+--------+
| level|          timestamp|          block_ha

30000

In [ ]:
df_transaction_operation.printSchema()
df_transaction_operation.show(5)
df_transaction_operation.count()

root
 |-- level: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- signature: string (nullable = true)
 |-- operation_hash: string (nullable = true)
 |-- operation_group_index: integer (nullable = true)
 |-- operation_index: integer (nullable = true)
 |-- content_index: integer (nullable = true)
 |-- internal_operation_index: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- fee: integer (nullable = true)
 |-- amount: long (nullable = true)
 |-- counter: integer (nullable = true)
 |-- gas_limit: integer (nullable = true)
 |-- storage_limit: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- consumed_gas: integer (nullable = true)
 |-- storage_size: integer (nullable = true)
 |-- parameters: string (nullable = true)

+------+-------------------+--------------------+--------------------+-------------------

20698

In [ ]:
from pyspark.sql.functions import *

tables = [
    df_transaction_operation,
    df_delegation_operation,
    df_balance_updates,
    df_blocks,
    df_operation
]

tables = [
    df.withColumn("timestamp", to_timestamp("timestamp"))
    for df in tables
]


In [ ]:
df_transaction_operation = df_transaction_operation.dropna(
    subset=["fee", "gas_limit", "status"]
)


In [ ]:
df_delegation_operation = df_delegation_operation.dropna(
    subset=["fee", "status"]
)


In [ ]:
df_operation.groupBy("operation_kind").count().show()


+--------------+-----+
|operation_kind|count|
+--------------+-----+
|   transaction|28860|
+--------------+-----+



In [ ]:
df_transaction_operation.groupBy("status").count().show()


+-----------+-----+
|     status|count|
+-----------+-----+
|    skipped|  664|
|    applied|19555|
|     failed|   95|
|backtracked|  169|
+-----------+-----+



In [ ]:
df_transaction_operation.select(
    mean("fee"),
    min("fee"),
    max("fee"),
    stddev("fee")
).show()


+-----------------+--------+--------+-----------------+
|         avg(fee)|min(fee)|max(fee)|      stddev(fee)|
+-----------------+--------+--------+-----------------+
|1921.348679392667|       0| 1000000|17692.96118616634|
+-----------------+--------+--------+-----------------+



In [ ]:
df_balance_updates.select(
    sum("change").alias("total_balance_change"),
    avg("change").alias("avg_change")
).show()


+--------------------+----------------+
|total_balance_change|      avg_change|
+--------------------+----------------+
|      -1898596660863|-6.32865553621E7|
+--------------------+----------------+



In [ ]:
df_blocks.select(
    avg("number_of_operations"),
    max("number_of_operations")
).show()


+-------------------------+-------------------------+
|avg(number_of_operations)|max(number_of_operations)|
+-------------------------+-------------------------+
|       154.39769396130546|                     2994|
+-------------------------+-------------------------+



In [ ]:
ml_df = df_transaction_operation.select(
    "fee",
    "gas_limit",
    "storage_limit",
    "counter",
    "status"
)

In [ ]:
ml_df = ml_df.withColumn(
    "label",
    when(col("status") == "applied", 1).otherwise(0)
)

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["fee", "gas_limit", "storage_limit", "counter"],
    outputCol="features"
)

final_df = assembler.transform(ml_df).select("features", "label")

In [ ]:
train_df, test_df = final_df.randomSplit([0.7, 0.3], seed=42)


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
lr_model = lr.fit(train_df)
lr_pred = lr_model.transform(test_df)


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(numTrees=50)
rf_model = rf.fit(train_df)
rf_pred = rf_model.transform(test_df)


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt_model = dt.fit(train_df)
dt_pred = dt_model.transform(test_df)


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

print("Logistic Regression:", evaluator.evaluate(lr_pred))
print("Random Forest:", evaluator.evaluate(rf_pred))
print("Decision Tree:", evaluator.evaluate(dt_pred))


Logistic Regression: 0.8254605218716818
Random Forest: 0.98629937076137
Decision Tree: 0.9123201788429504


In [ ]:
from pyspark.sql.functions import *

ml_df = df_transaction_operation.select(
    "fee",
    "gas_limit",
    "storage_limit",
    "counter",
    "status"
)

ml_df = ml_df.withColumn(
    "label",
    when(col("status") == "applied", 1).otherwise(0)
)


In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["fee", "gas_limit", "storage_limit", "counter"],
    outputCol="features"
)

final_df = assembler.transform(ml_df).select("features", "label")


In [ ]:
train_df, test_df = final_df.randomSplit([0.7, 0.3], seed=42)


In [ ]:
def confusion_matrix(predictions):
    return predictions.groupBy("label", "prediction") \
        .count() \
        .orderBy("label", "prediction")


In [ ]:
def classification_metrics(predictions):
    tp = predictions.filter("label = 1 AND prediction = 1").count()
    tn = predictions.filter("label = 0 AND prediction = 0").count()
    fp = predictions.filter("label = 0 AND prediction = 1").count()
    fn = predictions.filter("label = 1 AND prediction = 0").count()

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
lr_model = lr.fit(train_df)
lr_pred = lr_model.transform(test_df)


In [ ]:
confusion_matrix(lr_pred).show()


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|    4|
|    0|       1.0|  270|
|    1|       0.0|    2|
|    1|       1.0| 5725|
+-----+----------+-----+



In [ ]:
lr_metrics = classification_metrics(lr_pred)
print("Logistic Regression")
print("Accuracy:", lr_metrics[0])
print("Precision:", lr_metrics[1])
print("Recall:", lr_metrics[2])
print("F1-score:", lr_metrics[3])


Logistic Regression
Accuracy: 0.9546742209631728
Precision: 0.9549624687239366
Recall: 0.999650777021128
F1-score: 0.9767957686401638


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(maxDepth=5)
dt_model = dt.fit(train_df)
dt_pred = dt_model.transform(test_df)


In [ ]:
confusion_matrix(dt_pred).show()


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|  205|
|    0|       1.0|   69|
|    1|       0.0|    5|
|    1|       1.0| 5722|
+-----+----------+-----+



In [ ]:
dt_metrics = classification_metrics(dt_pred)
print("Decision Tree")
print("Accuracy:", dt_metrics[0])
print("Precision:", dt_metrics[1])
print("Recall:", dt_metrics[2])
print("F1-score:", dt_metrics[3])


Decision Tree
Accuracy: 0.9876687218796867
Precision: 0.9880849594197894
Recall: 0.99912694255282
F1-score: 0.9935752734849801


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(numTrees=50, maxDepth=7)
rf_model = rf.fit(train_df)
rf_pred = rf_model.transform(test_df)


In [ ]:
confusion_matrix(rf_pred).show()


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|  207|
|    0|       1.0|   67|
|    1|       0.0|    1|
|    1|       1.0| 5726|
+-----+----------+-----+



In [ ]:
rf_metrics = classification_metrics(rf_pred)
print("Random Forest")
print("Accuracy:", rf_metrics[0])
print("Precision:", rf_metrics[1])
print("Recall:", rf_metrics[2])
print("F1-score:", rf_metrics[3])


Random Forest
Accuracy: 0.9886685552407932
Precision: 0.9884343172794753
Recall: 0.999825388510564
F1-score: 0.9940972222222222


Logistic Regression: 0.8255143710353946
Random Forest: 0.9875713581077723
Decision Tree: 0.9123182670383214
